# SFunctor Demo: Structure Function Analysis for MHD Turbulence

This notebook provides a comprehensive demonstration of the SFunctor pipeline for analyzing anisotropic, angle-resolved structure functions from 2D slices of 3D magnetohydrodynamic (MHD) simulations.

## Overview

SFunctor is designed to:
- Extract 2D slices from 3D MHD simulation data
- Compute various physics quantities (velocity, magnetic field, density, Elsasser variables)
- Calculate structure functions with different displacement vectors
- Visualize turbulence statistics and scaling behaviors

This demo will walk you through:
1. Loading and exploring MHD simulation slices
2. Running structure function analysis (non-MPI version)
3. Creating comprehensive visualizations
4. Understanding the physics and computational pipeline

## 1. Setup and Imports

In [ ]:
# Standard library imports
import sys
from pathlib import Path
from datetime import datetime

# Scientific computing
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm
from matplotlib.gridspec import GridSpec

# Interactive widgets (optional, will use if available)
try:
    import ipywidgets as widgets
    from IPython.display import display
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("ipywidgets not available. Interactive features will be limited.")

# SFunctor modules
from sf_io import load_slice_npz, parse_slice_metadata
from sf_physics import compute_vA, compute_z_plus_minus
from sf_displacements import find_ell_bin_edges, build_displacement_list

# Set matplotlib parameters for better plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['font.size'] = 10

print("Setup complete!")
print(f"Working directory: {Path.cwd()}")

## 2. Check Available Data

In [ ]:
# Check what data files are available
data_dir = Path("slice_data")
if not data_dir.exists():
    print(f"Error: {data_dir} directory not found!")
    print("Please make sure you're running this notebook from the SFunctor root directory.")
else:
    # List available slice files
    slice_files = list(data_dir.glob("slice_*.npz"))
    slice_files = [f for f in slice_files if not f.stem.startswith("simple_sf_")]
    
    print(f"Found {len(slice_files)} slice files:")
    for f in sorted(slice_files):
        print(f"  - {f.name}")
    
    # List available structure function results
    sf_files = list(data_dir.glob("simple_sf_*.npz"))
    if sf_files:
        print(f"\nFound {len(sf_files)} pre-computed structure function results:")
        for f in sorted(sf_files):
            print(f"  - {f.name}")

In [ ]:
# Example: Create a slice list file for batch processing
slice_list_file = data_dir / "my_slices.txt"

# Gather all available slices
all_slice_files = list(data_dir.glob("slice_*.npz"))
all_slice_files = [f for f in all_slice_files if not f.stem.startswith("simple_sf_")]

if all_slice_files:
    # Write slice paths to a list file
    with open(slice_list_file, 'w') as f:
        for slice_path in sorted(all_slice_files):
            f.write(str(slice_path) + '\n')
    
    print(f"Created slice list file: {slice_list_file}")
    print(f"Contains {len(all_slice_files)} slices")
    
    # Show example of how to process all slices
    print("\nTo process all slices with structure function analysis:")
    print(f"python run_sf.py --slice_list {slice_list_file} --stride 2")
    
    # Or for MPI parallel processing:
    print("\nFor parallel processing with MPI:")
    print(f"mpirun -n 8 python run_sf.py --slice_list {slice_list_file} --stride 2")
else:
    print("No slice files available for batch processing.")

### Batch Processing Multiple Slices

For systematic analysis, you often want to extract many slices. Here's how to create a batch of slices and prepare them for structure function analysis:

In [ ]:
# Alternative: Direct Python extraction (if you prefer not to use subprocess)
# This shows how to extract a single slice programmatically

if raw_data_dir.exists() and sim_dirs:
    # Import the extraction function
    try:
        from extract_2d_slice import extract_2d_slice
        
        sim_name = sim_dirs[0].name.replace("data_", "")
        
        print("Extracting a single slice using Python function directly...")
        
        # Extract one slice as an example
        slice_data_direct = extract_2d_slice(
            sim_name=sim_name,
            axis=3,            # z-normal slice
            slice_value=0.0,   # at the midplane
            file_number=0,     # first snapshot
            save=True,         # save to cache
            cache_dir="slice_data"
        )
        
        print(f"\nExtracted slice with shape: {slice_data_direct['dens'].shape}")
        print(f"Available fields: {list(slice_data_direct.keys())}")
        
        # Quick visualization of the extracted density
        fig, ax = plt.subplots(figsize=(8, 8))
        im = ax.imshow(slice_data_direct['dens'], origin='lower', cmap='plasma')
        ax.set_title(f'Density Field - Direct Extraction\n{sim_name} (z=0 slice)')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.colorbar(im, ax=ax, label='Density')
        plt.tight_layout()
        plt.show()
        
    except ImportError as e:
        print(f"Could not import extract_2d_slice: {e}")
        print("The extraction module may have dependencies that are not available.")
        
    except Exception as e:
        print(f"Error during direct extraction: {e}")
        print("This may be due to the specific format of your simulation data.")

### Understanding Slice Extraction Parameters

The slice extraction tool has several important parameters:

- **sim_name**: The simulation directory name (without the `data_` prefix)
- **axes**: Which axes to slice along (1=x, 2=y, 3=z)
- **offsets**: Positions along each axis as fractions of box size (-0.5 to 0.5)
- **file_number**: Which snapshot to use (0 = first, None = latest)
- **plot**: Whether to generate PNG visualizations of the density field

The extracted slices contain:
- Primary fields: `dens`, `velx`, `vely`, `velz`, `bcc1`, `bcc2`, `bcc3`
- Derived fields: `vortx`, `vorty`, `vortz` (vorticity), `currx`, `curry`, `currz` (current density)

In [ ]:
# Example: Extract slices using subprocess (as done in demo_pipeline.py)
import subprocess

# Only run if we have simulation data
if raw_data_dir.exists() and sim_dirs:
    # Use the first available simulation
    sim_name = sim_dirs[0].name.replace("data_", "")
    
    print(f"Extracting slices from simulation: {sim_name}")
    print("\nThis will extract:")
    print("  - 2 slice orientations (x-normal and z-normal)")
    print("  - 3 positions along each axis (-0.25, 0.0, 0.25)")
    print("  - Total: 6 slices\n")
    
    # Build the command
    cmd = [
        sys.executable,
        "run_extract_slice.py",
        "--sim_name", sim_name,
        "--offsets", "-0.25,0.0,0.25",   # Three positions
        "--axes", "1,3",                  # x and z normal slices
        "--file_number", "0",             # Use first snapshot
        "--plot",                         # Generate density plots
    ]
    
    print("Running command:")
    print(" ".join(cmd))
    print("\nThis may take a minute...\n")
    
    try:
        # Run the extraction
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print("Extraction successful!")
        print("\nOutput:")
        print(result.stdout)
        
        # Check what was created
        new_slices = list(data_dir.glob(f"slice_*_{sim_name}_*.npz"))
        if new_slices:
            print(f"\nCreated {len(new_slices)} new slice files:")
            for f in sorted(new_slices)[:6]:
                print(f"  - {f.name}")
                
    except subprocess.CalledProcessError as e:
        print(f"Error during extraction: {e}")
        print(f"Error output: {e.stderr}")
        
else:
    print("Skipping slice extraction demo (no simulation data found)")
    print("\nTo extract slices from your own data:")
    print("1. Place your 3D simulation data in data/data_YourSimName/")
    print("2. Run: python run_extract_slice.py --sim_name YourSimName --axes 1,2,3 --offsets -0.25,0,0.25")

### Using run_extract_slice.py

The primary tool for extracting slices is `run_extract_slice.py`. It can extract multiple slices at different positions and orientations in a single run.

In [ ]:
# Check available raw data
raw_data_dir = Path("data")
if raw_data_dir.exists():
    print("Available simulation data:")
    sim_dirs = list(raw_data_dir.glob("data_*"))
    for sim_dir in sim_dirs:
        sim_name = sim_dir.name.replace("data_", "")
        print(f"\n  Simulation: {sim_name}")
        
        # Check for binary files
        bin_files = list(sim_dir.glob("bin/rank_*/Turb.*.bin"))
        if bin_files:
            print(f"    - Found {len(bin_files)} binary data files")
            # Show a few example files
            for f in sorted(bin_files)[:3]:
                print(f"      {f.relative_to(raw_data_dir)}")
            if len(bin_files) > 3:
                print(f"      ... and {len(bin_files) - 3} more files")
else:
    print("No raw data directory found.")
    print("To use slice extraction, place your 3D simulation data in a 'data/' directory.")

## 2.5. Extract 2D Slices from 3D Data

Before analyzing structure functions, we need to extract 2D slices from 3D simulation data. SFunctor provides tools to extract slices at various positions and orientations.

## 3. Load and Explore a 2D Slice

Let's load one of the example slices and explore its contents.

In [ ]:
# Select a slice file to analyze
if slice_files:
    slice_file = slice_files[0]  # Use the first available slice
    print(f"Loading slice: {slice_file}")
    
    # Load the slice data
    slice_data = load_slice_npz(slice_file, stride=1)  # stride=1 for full resolution
    
    # Parse metadata from filename
    axis, beta = parse_slice_metadata(slice_file)
    
    # Extract fields
    rho = slice_data["rho"]
    v_x = slice_data["v_x"]
    v_y = slice_data["v_y"]
    v_z = slice_data["v_z"]
    B_x = slice_data["B_x"]
    B_y = slice_data["B_y"]
    B_z = slice_data["B_z"]
    
    print(f"\nSlice properties:")
    print(f"  - Shape: {rho.shape}")
    print(f"  - Slice axis: {axis}")
    print(f"  - Plasma beta: {beta}")
    print(f"\nField statistics:")
    print(f"  - Density: min={rho.min():.3f}, max={rho.max():.3f}, mean={rho.mean():.3f}")
    print(f"  - |v|: min={np.sqrt(v_x**2 + v_y**2 + v_z**2).min():.3f}, max={np.sqrt(v_x**2 + v_y**2 + v_z**2).max():.3f}")
    print(f"  - |B|: min={np.sqrt(B_x**2 + B_y**2 + B_z**2).min():.3f}, max={np.sqrt(B_x**2 + B_y**2 + B_z**2).max():.3f}")
else:
    print("No slice files found!")

## 4. Visualize the Slice Fields

Let's create visualizations of the primary fields in the slice.

In [ ]:
# Create a comprehensive visualization of the slice
fig = plt.figure(figsize=(15, 10))
gs = GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# Helper function for field plots
def plot_field(ax, field, title, cmap='viridis', norm=None):
    im = ax.imshow(field, origin='lower', cmap=cmap, norm=norm)
    ax.set_title(title)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.colorbar(im, ax=ax, fraction=0.046)
    return im

# Density
ax1 = fig.add_subplot(gs[0, 0])
plot_field(ax1, rho, 'Density ρ', cmap='plasma')

# Velocity magnitude
ax2 = fig.add_subplot(gs[0, 1])
v_mag = np.sqrt(v_x**2 + v_y**2 + v_z**2)
plot_field(ax2, v_mag, 'Velocity Magnitude |v|', cmap='viridis')

# Magnetic field magnitude
ax3 = fig.add_subplot(gs[0, 2])
B_mag = np.sqrt(B_x**2 + B_y**2 + B_z**2)
plot_field(ax3, B_mag, 'Magnetic Field Magnitude |B|', cmap='inferno')

# Velocity components
ax4 = fig.add_subplot(gs[1, 0])
plot_field(ax4, v_x, 'Velocity vₓ', cmap='RdBu_r', norm=SymLogNorm(linthresh=0.01))

ax5 = fig.add_subplot(gs[1, 1])
plot_field(ax5, v_y, 'Velocity vᵧ', cmap='RdBu_r', norm=SymLogNorm(linthresh=0.01))

ax6 = fig.add_subplot(gs[1, 2])
plot_field(ax6, v_z, 'Velocity vᵤ', cmap='RdBu_r', norm=SymLogNorm(linthresh=0.01))

# Compute and plot derived quantities
# Alfvén velocity
vA_x, vA_y, vA_z = compute_vA(B_x, B_y, B_z, rho)
vA_mag = np.sqrt(vA_x**2 + vA_y**2 + vA_z**2)

ax7 = fig.add_subplot(gs[2, 0])
plot_field(ax7, vA_mag, 'Alfvén Velocity |vₐ|', cmap='copper')

# Elsasser variables
(zp_x, zp_y, zp_z), (zm_x, zm_y, zm_z) = compute_z_plus_minus(v_x, v_y, v_z, vA_x, vA_y, vA_z)
zp_mag = np.sqrt(zp_x**2 + zp_y**2 + zp_z**2)
zm_mag = np.sqrt(zm_x**2 + zm_y**2 + zm_z**2)

ax8 = fig.add_subplot(gs[2, 1])
plot_field(ax8, zp_mag, 'Elsasser z⁺ = v + vₐ', cmap='spring')

ax9 = fig.add_subplot(gs[2, 2])
plot_field(ax9, zm_mag, 'Elsasser z⁻ = v - vₐ', cmap='autumn')

plt.suptitle(f'MHD Slice Visualization: {slice_file.name}', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Simple Structure Function Analysis

Now let's perform a simplified structure function analysis. This version doesn't use MPI or Numba, making it easier to understand and modify.

In [ ]:
# Define the simplified structure function computation
def compute_structure_functions_simple(
    v_x, v_y, v_z, B_x, B_y, B_z, rho,
    delta_i, delta_j, N_random_subsamples=100
):
    """Simplified structure function computation."""
    N, M = v_x.shape
    
    # Random spatial samples
    flat_indices = np.random.choice(M * N, size=N_random_subsamples, replace=False)
    random_points_y = flat_indices // M
    random_points_x = flat_indices % M
    
    # Storage for structure function values
    dv_values = []
    dB_values = []
    drho_values = []
    
    # Also store individual components for more detailed analysis
    dv_components = {'x': [], 'y': [], 'z': []}
    dB_components = {'x': [], 'y': [], 'z': []}
    
    for idx in range(N_random_subsamples):
        i = random_points_x[idx]
        j = random_points_y[idx]
        ip = (i + delta_i) % M
        jp = (j + delta_j) % N
        
        # Compute differences using 2-point stencil
        dvx = v_x[jp, ip] - v_x[j, i]
        dvy = v_y[jp, ip] - v_y[j, i]
        dvz = v_z[jp, ip] - v_z[j, i]
        
        dBx = B_x[jp, ip] - B_x[j, i]
        dBy = B_y[jp, ip] - B_y[j, i]
        dBz = B_z[jp, ip] - B_z[j, i]
        
        drho = rho[jp, ip] - rho[j, i]
        
        # Store components
        dv_components['x'].append(dvx)
        dv_components['y'].append(dvy)
        dv_components['z'].append(dvz)
        dB_components['x'].append(dBx)
        dB_components['y'].append(dBy)
        dB_components['z'].append(dBz)
        
        # Compute magnitudes
        dv = np.sqrt(dvx**2 + dvy**2 + dvz**2)
        dB = np.sqrt(dBx**2 + dBy**2 + dBz**2)
        drho_abs = abs(drho)
        
        dv_values.append(dv)
        dB_values.append(dB)
        drho_values.append(drho_abs)
    
    return (np.array(dv_values), np.array(dB_values), np.array(drho_values),
            {k: np.array(v) for k, v in dv_components.items()},
            {k: np.array(v) for k, v in dB_components.items()})

In [ ]:
# Set up displacement vectors
N_res = rho.shape[0]
ell_max = N_res // 4  # Conservative choice to avoid boundary effects
n_ell_bins = 10  # Number of logarithmic bins for displacement magnitude
n_disp_total = 50  # Total number of displacement vectors to sample

# Generate displacement bins
ell_bin_edges = find_ell_bin_edges(1.0, ell_max, n_ell_bins)
displacements = build_displacement_list(ell_bin_edges, n_disp_total)

print(f"Grid size: {N_res}x{N_res}")
print(f"Maximum displacement: {ell_max} pixels")
print(f"Number of displacement bins: {n_ell_bins}")
print(f"Total displacement vectors: {len(displacements)}")
print(f"\nFirst few displacements:")
for i, (dx, dy) in enumerate(displacements[:5]):
    r = np.sqrt(dx**2 + dy**2)
    print(f"  {i}: (Δx={dx:6.2f}, Δy={dy:6.2f}) → r={r:6.2f}")

In [ ]:
# Run the structure function analysis
print("Computing structure functions...")
print("(This may take a minute for large grids)\n")

# Storage for all results
all_displacements = []
all_dv_values = []
all_dB_values = []
all_drho_values = []
all_dv_components = {'x': [], 'y': [], 'z': []}
all_dB_components = {'x': [], 'y': [], 'z': []}

# Process a subset of displacements for demonstration
n_disp_demo = min(20, len(displacements))  # Limit for faster execution
N_random_subsamples = 200  # Number of random points per displacement

for disp_idx, (dx, dy) in enumerate(displacements[:n_disp_demo]):
    if disp_idx % 5 == 0:
        print(f"Processing displacement {disp_idx+1}/{n_disp_demo}...")
    
    # Compute structure functions for this displacement
    dv_vals, dB_vals, drho_vals, dv_comp, dB_comp = compute_structure_functions_simple(
        v_x, v_y, v_z, B_x, B_y, B_z, rho,
        int(dx), int(dy), N_random_subsamples
    )
    
    # Store displacement magnitude
    r = np.sqrt(dx**2 + dy**2)
    all_displacements.extend([r] * len(dv_vals))
    all_dv_values.extend(dv_vals)
    all_dB_values.extend(dB_vals)
    all_drho_values.extend(drho_vals)
    
    # Store components
    for comp in ['x', 'y', 'z']:
        all_dv_components[comp].extend(dv_comp[comp])
        all_dB_components[comp].extend(dB_comp[comp])

# Convert to arrays
all_displacements = np.array(all_displacements)
all_dv_values = np.array(all_dv_values)
all_dB_values = np.array(all_dB_values)
all_drho_values = np.array(all_drho_values)
for comp in ['x', 'y', 'z']:
    all_dv_components[comp] = np.array(all_dv_components[comp])
    all_dB_components[comp] = np.array(all_dB_components[comp])

print(f"\nAnalysis complete!")
print(f"Total samples: {len(all_dv_values)}")
print(f"Velocity SF range: {all_dv_values.min():.6f} - {all_dv_values.max():.6f}")
print(f"Magnetic SF range: {all_dB_values.min():.6f} - {all_dB_values.max():.6f}")
print(f"Density SF range: {all_drho_values.min():.6f} - {all_drho_values.max():.6f}")

## 6. Comprehensive Structure Function Visualization

Now let's create various visualizations to analyze the structure functions.

In [ ]:
# Bin the data by displacement
r_bins = np.logspace(np.log10(all_displacements.min()), 
                     np.log10(all_displacements.max()), 15)
r_centers = 0.5 * (r_bins[1:] + r_bins[:-1])

# Compute binned statistics
def bin_statistics(displacements, values, bins):
    """Compute mean and std in each bin."""
    means = []
    stds = []
    counts = []
    
    for i in range(len(bins) - 1):
        mask = (displacements >= bins[i]) & (displacements < bins[i+1])
        if np.sum(mask) > 0:
            means.append(np.mean(values[mask]))
            stds.append(np.std(values[mask]))
            counts.append(np.sum(mask))
        else:
            means.append(np.nan)
            stds.append(np.nan)
            counts.append(0)
    
    return np.array(means), np.array(stds), np.array(counts)

# Compute binned statistics for each field
dv_mean, dv_std, dv_counts = bin_statistics(all_displacements, all_dv_values, r_bins)
dB_mean, dB_std, dB_counts = bin_statistics(all_displacements, all_dB_values, r_bins)
drho_mean, drho_std, drho_counts = bin_statistics(all_displacements, all_drho_values, r_bins)

# Remove bins with no data
valid = ~np.isnan(dv_mean)
r_centers_valid = r_centers[valid]
dv_mean_valid = dv_mean[valid]
dB_mean_valid = dB_mean[valid]
drho_mean_valid = drho_mean[valid]

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(16, 12))
gs = GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# 1. Structure functions vs displacement (with theoretical scalings)
ax1 = fig.add_subplot(gs[0, :])
ax1.loglog(r_centers_valid, dv_mean_valid, 'o-', label='Velocity |δv|', color='blue', markersize=8)
ax1.loglog(r_centers_valid, dB_mean_valid, 's-', label='Magnetic |δB|', color='red', markersize=8)
ax1.loglog(r_centers_valid, drho_mean_valid, '^-', label='Density |δρ|', color='green', markersize=8)

# Add theoretical scaling lines
r_theory = np.linspace(r_centers_valid.min(), r_centers_valid.max(), 100)

# Kolmogorov scaling for velocity (1/3)
idx_mid = len(r_centers_valid)//2
C_v = dv_mean_valid[idx_mid] / (r_centers_valid[idx_mid]**(1/3))
ax1.loglog(r_theory, C_v * r_theory**(1/3), '--', alpha=0.5, color='blue', label='r^(1/3)')

# Iroshnikov-Kraichnan scaling (1/2)
C_B = dB_mean_valid[idx_mid] / (r_centers_valid[idx_mid]**(1/2))
ax1.loglog(r_theory, C_B * r_theory**(1/2), '--', alpha=0.5, color='red', label='r^(1/2)')

# Linear scaling
C_rho = drho_mean_valid[idx_mid] / r_centers_valid[idx_mid]
ax1.loglog(r_theory, C_rho * r_theory, '--', alpha=0.5, color='green', label='r^1')

ax1.set_xlabel('Displacement r [pixels]', fontsize=12)
ax1.set_ylabel('Structure Function', fontsize=12)
ax1.set_title('Structure Functions with Theoretical Scalings', fontsize=14)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# 2. Probability distributions
ax2 = fig.add_subplot(gs[1, 0])
ax2.hist(all_dv_values, bins=50, alpha=0.7, density=True, color='blue', edgecolor='black')
ax2.set_xlabel('|δv|')
ax2.set_ylabel('PDF')
ax2.set_title('Velocity Increment Distribution')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

ax3 = fig.add_subplot(gs[1, 1])
ax3.hist(all_dB_values, bins=50, alpha=0.7, density=True, color='red', edgecolor='black')
ax3.set_xlabel('|δB|')
ax3.set_ylabel('PDF')
ax3.set_title('Magnetic Increment Distribution')
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3)

ax4 = fig.add_subplot(gs[1, 2])
ax4.hist(all_drho_values, bins=50, alpha=0.7, density=True, color='green', edgecolor='black')
ax4.set_xlabel('|δρ|')
ax4.set_ylabel('PDF')
ax4.set_title('Density Increment Distribution')
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

# 3. Component analysis
ax5 = fig.add_subplot(gs[2, 0])
components = ['x', 'y', 'z']
colors = ['red', 'green', 'blue']
for comp, color in zip(components, colors):
    ax5.hist(all_dv_components[comp], bins=50, alpha=0.5, density=True, 
             label=f'δv_{comp}', color=color, edgecolor='black')
ax5.set_xlabel('δv components')
ax5.set_ylabel('PDF')
ax5.set_title('Velocity Component Distributions')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 4. Scatter plot of velocity vs magnetic increments
ax6 = fig.add_subplot(gs[2, 1])
# Sample points for clarity
sample_idx = np.random.choice(len(all_dv_values), size=min(1000, len(all_dv_values)), replace=False)
ax6.scatter(all_dv_values[sample_idx], all_dB_values[sample_idx], 
            alpha=0.5, s=20, c=all_displacements[sample_idx], cmap='viridis')
cbar = plt.colorbar(ax6.scatter([], [], c=[], cmap='viridis'), ax=ax6)
cbar.set_label('Displacement r')
ax6.set_xlabel('|δv|')
ax6.set_ylabel('|δB|')
ax6.set_title('Velocity vs Magnetic Increments')
ax6.grid(True, alpha=0.3)

# 5. Local slopes (scaling exponents)
ax7 = fig.add_subplot(gs[2, 2])
if len(r_centers_valid) > 3:
    # Compute local slopes
    def local_slope(x, y):
        """Compute local slope in log-log space."""
        log_x = np.log10(x)
        log_y = np.log10(y)
        slopes = np.diff(log_y) / np.diff(log_x)
        x_mid = 0.5 * (x[1:] + x[:-1])
        return x_mid, slopes
    
    r_mid_v, slopes_v = local_slope(r_centers_valid, dv_mean_valid)
    r_mid_B, slopes_B = local_slope(r_centers_valid, dB_mean_valid)
    r_mid_rho, slopes_rho = local_slope(r_centers_valid, drho_mean_valid)
    
    ax7.semilogx(r_mid_v, slopes_v, 'o-', label='Velocity', color='blue')
    ax7.semilogx(r_mid_B, slopes_B, 's-', label='Magnetic', color='red')
    ax7.semilogx(r_mid_rho, slopes_rho, '^-', label='Density', color='green')
    
    # Add reference lines
    ax7.axhline(1/3, linestyle='--', alpha=0.5, color='blue', label='1/3')
    ax7.axhline(1/2, linestyle='--', alpha=0.5, color='red', label='1/2')
    ax7.axhline(1, linestyle='--', alpha=0.5, color='green', label='1')
    
    ax7.set_xlabel('Displacement r [pixels]')
    ax7.set_ylabel('Local Scaling Exponent')
    ax7.set_title('Scale-Dependent Exponents')
    ax7.legend(loc='best', fontsize=9)
    ax7.grid(True, alpha=0.3)
    ax7.set_ylim(0, 1.2)

plt.suptitle(f'Structure Function Analysis: {slice_file.name}', fontsize=16)
plt.tight_layout()
plt.show()

## 7. Elsasser Variable Structure Functions

Let's also compute structure functions for the Elsasser variables, which are important for understanding MHD turbulence.

In [ ]:
# Compute structure functions for Elsasser variables
print("Computing Elsasser variable structure functions...")

# Compute Alfvén velocity and Elsasser variables
vA_x, vA_y, vA_z = compute_vA(B_x, B_y, B_z, rho)
(zp_x, zp_y, zp_z), (zm_x, zm_y, zm_z) = compute_z_plus_minus(v_x, v_y, v_z, vA_x, vA_y, vA_z)

# Storage for Elsasser structure functions
all_dzp_values = []
all_dzm_values = []

# Compute for a subset of displacements
for disp_idx, (dx, dy) in enumerate(displacements[:n_disp_demo]):
    if disp_idx % 5 == 0:
        print(f"Processing displacement {disp_idx+1}/{n_disp_demo}...")
    
    # Use the same function but with Elsasser variables
    dzp_vals, dzm_vals, _, _, _ = compute_structure_functions_simple(
        zp_x, zp_y, zp_z, zm_x, zm_y, zm_z, rho,  # Note: using z- in place of B for simplicity
        int(dx), int(dy), N_random_subsamples
    )
    
    all_dzp_values.extend(dzp_vals)
    all_dzm_values.extend(dzm_vals)

all_dzp_values = np.array(all_dzp_values)
all_dzm_values = np.array(all_dzm_values)

print(f"\nElsasser z⁺ SF range: {all_dzp_values.min():.6f} - {all_dzp_values.max():.6f}")
print(f"Elsasser z⁻ SF range: {all_dzm_values.min():.6f} - {all_dzm_values.max():.6f}")

In [ ]:
# Bin Elsasser structure functions
dzp_mean, dzp_std, _ = bin_statistics(all_displacements, all_dzp_values, r_bins)
dzm_mean, dzm_std, _ = bin_statistics(all_displacements, all_dzm_values, r_bins)

# Plot comparison of all structure functions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# All structure functions together
ax1.loglog(r_centers_valid, dv_mean_valid, 'o-', label='|δv|', color='blue', markersize=8)
ax1.loglog(r_centers_valid, dB_mean_valid, 's-', label='|δB|', color='red', markersize=8)
ax1.loglog(r_centers_valid, drho_mean_valid, '^-', label='|δρ|', color='green', markersize=8)
ax1.loglog(r_centers[valid], dzp_mean[valid], 'd-', label='|δz⁺|', color='purple', markersize=8)
ax1.loglog(r_centers[valid], dzm_mean[valid], 'v-', label='|δz⁻|', color='orange', markersize=8)

ax1.set_xlabel('Displacement r [pixels]', fontsize=12)
ax1.set_ylabel('Structure Function', fontsize=12)
ax1.set_title('All Structure Functions', fontsize=14)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Elsasser variable asymmetry
ax2.semilogx(r_centers[valid], dzp_mean[valid] / dzm_mean[valid], 'o-', color='black', markersize=8)
ax2.axhline(1, linestyle='--', color='gray', alpha=0.5)
ax2.set_xlabel('Displacement r [pixels]', fontsize=12)
ax2.set_ylabel('|δz⁺| / |δz⁻|', fontsize=12)
ax2.set_title('Elsasser Variable Asymmetry', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0.5, 2.0)

plt.tight_layout()
plt.show()

## 8. Interactive Parameter Exploration

If ipywidgets is available, let's create an interactive tool to explore how parameters affect the analysis.

In [ ]:
if HAS_WIDGETS:
    def interactive_sf_analysis(stride=1, n_samples=100, max_disp_fraction=0.25):
        """Interactive structure function analysis with adjustable parameters."""
        
        # Reload data with new stride
        slice_data_int = load_slice_npz(slice_file, stride=stride)
        rho_int = slice_data_int["rho"]
        v_x_int = slice_data_int["v_x"]
        v_y_int = slice_data_int["v_y"]
        v_z_int = slice_data_int["v_z"]
        B_x_int = slice_data_int["B_x"]
        B_y_int = slice_data_int["B_y"]
        B_z_int = slice_data_int["B_z"]
        
        # Set up displacements
        N_res_int = rho_int.shape[0]
        ell_max_int = int(N_res_int * max_disp_fraction)
        
        # Simple displacement selection for interactive use
        displacements_int = [(i, 0) for i in range(1, ell_max_int, max(1, ell_max_int//10))]
        
        # Compute structure functions
        r_vals = []
        dv_vals = []
        dB_vals = []
        
        for dx, dy in displacements_int:
            dv, dB, drho, _, _ = compute_structure_functions_simple(
                v_x_int, v_y_int, v_z_int, B_x_int, B_y_int, B_z_int, rho_int,
                dx, dy, n_samples
            )
            r_vals.append(np.sqrt(dx**2 + dy**2))
            dv_vals.append(np.mean(dv))
            dB_vals.append(np.mean(dB))
        
        # Plot results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Structure functions
        ax1.loglog(r_vals, dv_vals, 'o-', label='Velocity', color='blue', markersize=8)
        ax1.loglog(r_vals, dB_vals, 's-', label='Magnetic', color='red', markersize=8)
        ax1.set_xlabel('Displacement r')
        ax1.set_ylabel('Structure Function')
        ax1.set_title(f'Structure Functions (stride={stride}, samples={n_samples})')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Field visualization
        v_mag_int = np.sqrt(v_x_int**2 + v_y_int**2 + v_z_int**2)
        im = ax2.imshow(v_mag_int, origin='lower', cmap='viridis')
        ax2.set_title(f'Velocity Magnitude (grid size: {N_res_int}x{N_res_int})')
        plt.colorbar(im, ax=ax2, fraction=0.046)
        
        plt.tight_layout()
        plt.show()
    
    # Create interactive widgets
    print("Interactive Structure Function Explorer")
    print("Adjust parameters to see their effect on the analysis:\n")
    
    interact = widgets.interactive(
        interactive_sf_analysis,
        stride=widgets.IntSlider(value=1, min=1, max=4, step=1, 
                                description='Stride:'),
        n_samples=widgets.IntSlider(value=100, min=50, max=500, step=50, 
                                   description='Samples:'),
        max_disp_fraction=widgets.FloatSlider(value=0.25, min=0.1, max=0.5, step=0.05,
                                             description='Max disp:')
    )
    display(interact)
else:
    print("Interactive features not available. Install ipywidgets for interactive exploration.")

## 9. Understanding the Full Pipeline

The complete SFunctor pipeline computes 24 different structure function channels. Let's explore what these are and how they relate to MHD turbulence physics.

In [ ]:
# Display information about the 24 channels
channel_info = [
    ("Basic Fields (9 channels):", [
        "Channel 0-2: Velocity components (δvₓ, δvᵧ, δvᵤ)",
        "Channel 3-5: Magnetic field components (δBₓ, δBᵧ, δBᵤ)",
        "Channel 6-8: Density-weighted velocity (δ(ρv)ₓ, δ(ρv)ᵧ, δ(ρv)ᵤ)"
    ]),
    ("Elsasser Variables (6 channels):", [
        "Channel 9-11: z⁺ = v + vₐ components",
        "Channel 12-14: z⁻ = v - vₐ components"
    ]),
    ("Derived Quantities (9 channels):", [
        "Channel 15-17: Vorticity ω = ∇ × v",
        "Channel 18-20: Current density J = ∇ × B",
        "Channel 21-23: Cross products and other correlations"
    ])
]

print("=" * 60)
print("24 Structure Function Channels in Full Pipeline")
print("=" * 60)

for category, channels in channel_info:
    print(f"\n{category}")
    for channel in channels:
        print(f"  • {channel}")

print("\n" + "=" * 60)
print("\nThese channels capture different aspects of MHD turbulence:")
print("- Basic fields reveal energy cascade in velocity and magnetic fields")
print("- Elsasser variables separate forward/backward Alfvén wave packets")
print("- Vorticity and current density probe small-scale structures")
print("- Cross correlations reveal coupling between fields")

## 10. Comparison: Simple vs Full Analysis

Let's compare the simplified analysis we just performed with what the full pipeline computes.

In [ ]:
comparison_data = {
    "Aspect": [
        "Parallelization",
        "Performance",
        "Channels computed",
        "Displacement sampling",
        "Stencil options",
        "Memory usage",
        "Output format",
        "Dependencies"
    ],
    "Simple Analysis (this notebook)": [
        "None (serial execution)",
        "Slower (no Numba JIT)",
        "3 (|δv|, |δB|, |δρ|)",
        "Random subset",
        "2-point only",
        "Low",
        "Simple arrays",
        "NumPy only"
    ],
    "Full Pipeline (run_sf.py)": [
        "MPI + shared memory",
        "Fast (Numba-optimized)",
        "24 (all components)",
        "Systematic coverage",
        "2, 3, or 5-point",
        "Higher",
        "Structured histograms",
        "NumPy, Numba, MPI4Py"
    ]
}

# Create comparison table
print("\nComparison: Simple vs Full Analysis")
print("=" * 80)
print(f"{'Aspect':<25} {'Simple (Demo)':<30} {'Full Pipeline':<25}")
print("=" * 80)

for i in range(len(comparison_data["Aspect"])):
    aspect = comparison_data["Aspect"][i]
    simple = comparison_data["Simple Analysis (this notebook)"][i]
    full = comparison_data["Full Pipeline (run_sf.py)"][i]
    print(f"{aspect:<25} {simple:<30} {full:<25}")

print("\n" + "=" * 80)
print("\nWhen to use each approach:")
print("- Simple: Quick exploration, testing, understanding the method")
print("- Full: Production runs, large datasets, complete statistics")

## 11. Save Analysis Results

Let's save our analysis results in a format compatible with the visualization tools.

In [ ]:
# Save the computed structure functions
output_file = Path("demo_sf_results.npz")

np.savez(
    output_file,
    # Basic structure function data
    displacements=all_displacements,
    dv_values=all_dv_values,
    dB_values=all_dB_values,
    drho_values=all_drho_values,
    
    # Component data
    dv_components=all_dv_components,
    dB_components=all_dB_components,
    
    # Elsasser variables
    dzp_values=all_dzp_values,
    dzm_values=all_dzm_values,
    
    # Binned statistics
    r_bins=r_bins,
    r_centers=r_centers,
    dv_mean=dv_mean,
    dB_mean=dB_mean,
    drho_mean=drho_mean,
    
    # Metadata
    meta=dict(
        slice=str(slice_file),
        axis=axis,
        beta=beta,
        n_samples=N_random_subsamples,
        n_displacements=n_disp_demo,
        date=datetime.utcnow().isoformat(),
        analysis_type='demo_notebook'
    )
)

print(f"Results saved to: {output_file}")
print(f"File size: {output_file.stat().st_size / 1024:.1f} KB")

## 12. Summary and Next Steps

### What We've Covered

In this notebook, we've demonstrated:

1. **Data Loading**: How to load and explore 2D slices from MHD simulations
2. **Field Visualization**: Creating comprehensive views of velocity, magnetic field, and density
3. **Structure Function Analysis**: Computing increments at different displacements
4. **Statistical Analysis**: Binning data and computing scaling behaviors
5. **Comprehensive Plotting**: Various ways to visualize structure functions
6. **Physics Insights**: Understanding Elsasser variables and MHD turbulence

### Next Steps

To use SFunctor for your own analysis:

1. **Extract slices from your data**:
   ```bash
   python extract_2d_slice.py --input_file your_3d_data.bin --output_file slice.npz
   ```

2. **Run the full pipeline** (with MPI for large datasets):
   ```bash
   mpirun -n 64 python run_sf.py --file_name slice.npz --stride 2
   ```

3. **Visualize results**:
   ```bash
   python visualize_sf_results.py results_sf.npz
   ```

### Key Physics Insights

- **Scaling laws**: Different fields show different scaling behaviors
  - Velocity: Often close to Kolmogorov (1/3) or steeper
  - Magnetic: Can show Iroshnikov-Kraichnan (1/2) scaling
  - Density: Often shows shallower scaling

- **Elsasser asymmetry**: The ratio |δz⁺|/|δz⁻| reveals the balance between forward and backward Alfvén waves

- **Intermittency**: Heavy tails in increment PDFs indicate intermittent structures

### References

For more on MHD turbulence and structure functions, see:
- Biskamp, D. (2003). Magnetohydrodynamic Turbulence
- Boldyrev, S. (2006). Spectrum of Magnetohydrodynamic Turbulence
- Beresnyak, A. & Lazarian, A. (2019). MHD Turbulence, Turbulent Dynamo and Applications

In [ ]:
print("\n" + "="*60)
print("Demo notebook complete!")
print("="*60)
print(f"\nAnalyzed slice: {slice_file.name if 'slice_file' in locals() else 'No slice loaded'}")
print(f"Results saved to: {output_file if 'output_file' in locals() else 'No results saved'}")
print("\nThank you for using SFunctor!")